In [5]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
from PIL import Image
import numpy as np
import os
import torch.nn.functional as F


transform = T.Compose([
#     T.Resize((227,227)), 
    T.ToTensor(),  # 将Image转成Tensor，归一化至 [0.0, 1.0]
#     T.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

class Codeimg(Dataset):
    
    def __init__(self, root, transform=None):
        self.root = root
        self.paths = os.listdir(root)
        self.transforms = transform   
        
    def one_hot(self, label):
        bp = torch.Tensor([])
        for i in range(len(label)):
            num = ord(label[i])-48
            if num>9:
                num -= 7
                if num>35:
                    num -= 6
            a = num
            b = torch.zeros(1, 62)
            b[:,a] = 1
            bp = torch.cat((bp,b),dim=1)
        return bp
        
    def __getitem__(self, index):
        image_path = self.paths[index]    
        label = list(image_path)[:-4]
        label = self.one_hot(label).reshape(310)

        pil_image = Image.open(self.root+image_path)
        if self.transforms:
            data = self.transforms(pil_image)
        else:
            image_array = np.asarray(pil_image)
            data = torch.from_numpy(image_array)
        return data, label

    def __len__(self):
        return len(self.paths)

In [6]:
class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )

    def forward(self, x):
        out = self.left(x)
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=310):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.layer1 = self.make_layer(ResidualBlock, 64,  2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)
#         self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=2)
        self.fc = nn.Linear(4608, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
#         out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

def ResNet18():
    return ResNet(ResidualBlock)

In [7]:
data = Codeimg('./train/train/', transform)
dataloader = DataLoader(data, batch_size=64, shuffle=True, drop_last=False)

# dataiter = iter(dataloader) #是可迭代对象，iter生成迭代器
# imgs, labels = next(dataiter) 
# i = torchvision.utils.make_grid(imgs, 8)
# to_img = T.ToPILImage() #展示图片
# to_img(i)

In [ ]:
cnn = ResNet18()
# loss_fn = nn.CrossEntropyLoss()
loss_fn = nn.MultiLabelSoftMarginLoss()
optimizer = torch.optim.Adam(cnn.parameters())

# cnn = cnn.cuda()

for i in range(2):  
    for j,(img,labels) in enumerate(dataloader):
#         img = img.cuda()
#         labels = labels.cuda()
        out = cnn(img)
        loss = loss_fn(out, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if j % 50 == 0:
            print('i=%d j=%d Loss: %.5f' %(i,j,loss.item()))

In [ ]:
class UnCodeimg(Dataset):
    
    def __init__(self, root, transform=None):
        self.root = root
        self.paths = os.listdir(root)
        self.transforms = transform   
        
    def __getitem__(self, index):
        image_path = self.paths[index]
        label = image_path
        if label !='.DS_Store':
            label = list(label)[:-4]
            label = int(''.join(label))
            pil_image = Image.open(self.root+image_path)   
            if self.transforms:
                data = self.transforms(pil_image)
            else:
                image_array = np.asarray(pil_image)
                data = torch.from_numpy(image_array)
            return data, label

    def __len__(self):
        return len(self.paths)

In [4]:
def uncode(code):
    biao = list()
    for i in range(len(code)):
        if code[i]<10:
            biao.append(chr(code[i]+48))
        elif 10<=code[i]<36:
            biao.append(chr(code[i]+55))
        else: 
            biao.append(chr(code[i]+61))
    return biao   

In [5]:
cnn = ResNet18()
cnn.load_state_dict(torch.load('can.pt', map_location='cpu'))
cnn.eval()
# dataloader = DataLoader(data, batch_size=32, shuffle=True, drop_last=False)
data = UnCodeimg('test/test/', transform)

In [6]:
cnn = cnn.cuda()
chu = dict()

len(data)
for i in range(len(data)):
    imgs, labels = data[i]
    imgs = torch.Tensor(imgs).reshape(1,3,30,150)
    imgs = imgs.cuda()
    output = cnn(imgs)
    output = output.view(-1, 62)
    output = nn.functional.softmax(output, dim=1)
    output = torch.argmax(output, dim=1)
    output = output.cpu()
    out = uncode(output)
    chu[labels] = out
    if i%1000==0:
        print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000


In [7]:
num = list()
labels = list()
for i in range(len(chu)):
    num.append(i)
    labels.append(''.join(chu[i]))

In [8]:
y = np.array(labels)
np.savetxt('yan.csv',y,fmt="%s")